In [ ]:
# works for pure text wala pdf 

# import PyPDF2

# pdfFileObj = open('/var/home/ramrshrcg/Downloads/Natural-language-processing-with-Transformers-_-building-Lewis-Tunstall-Leandro-von-Werra-Thomas-Wolf-Revised-edition-Sebastopol-CA-2022-9781098103170-0060a3b3e6eaec8c73a59ce5fe80bddf-Annas-Archiv.pdf', 'rb')

# pdfReader = PyPDF2.PdfReader(pdfFileObj)

# # num_pages = pdfReader.numPage
# # s
# num_pages = len(pdfReader.pages)

# text = ""

# # for i in range(num_pages):
    
#     # pageObj = pdfReader.pages[i]
#     # text += pageObj.extract_text()

# # print(text)

# pageObj = pdfReader.pages[2]
# text += pageObj.extract_text()
# print(text)

In [ ]:
# Just remove the --user flag
%pip install pdfplumber pytesseract pdf2image Pillow

In [ ]:
# import PyPDF2
# import pdfplumber
# import pytesseract
# from pdf2image import convert_from_path
# from PIL import Image
# import os
# import gc
# import sys
# import re

# class MemoryOptimizedPDFExtractor:
#     def clean_nepali_text(self, text):
#         """
#         Remove all English letters, numbers, and keep only Nepali/Devanagari text
#         """
#         if not text:
#             return ""
        
#         # Remove English letters (a-z, A-Z) and numbers (0-9)
#         # Keep Devanagari characters (0x0900-0x097F), whitespace, and common punctuation
#         cleaned_lines = []
        
#         for line in text.split('\n'):
#             # Remove English letters and numbers
#             cleaned_line = re.sub(r'[a-zA-Z0-9]', '', line)
            
#             # Remove extra whitespace
#             cleaned_line = ' '.join(cleaned_line.split())
            
#             # Only keep lines that have meaningful content after cleaning
#             if cleaned_line.strip():
#                 cleaned_lines.append(cleaned_line)
        
#         return '\n'.join(cleaned_lines)
    
#     def format_page_separator(self, page_number):
#         """Create page separator in Nepali numerals"""
#         # Convert to Nepali numerals
#         nepali_numerals = {'0': '०', '1': '१', '2': '२', '3': '३', '4': '४', 
#                           '5': '५', '6': '६', '7': '७', '8': '८', '9': '९'}
        
#         page_str = str(page_number)
#         nepali_page_num = ''.join(nepali_numerals.get(digit, digit) for digit in page_str)
        
#         return f"\n--- पृष्ठ {nepali_page_num} ---\n"
#     def __init__(self, max_pages_batch=5, dpi=200):
#         """
#         Memory-optimized PDF text extractor
        
#         Args:
#             max_pages_batch: Process this many pages at once (lower = less memory)
#             dpi: Image resolution (lower = less memory, but worse OCR quality)
#         """
#         self.max_pages_batch = max_pages_batch
#         self.dpi = dpi
        
#     def extract_text_lightweight(self, pdf_path):
#         """Lightweight text extraction - tries pdfplumber first"""
#         try:
#             text = ""
#             with pdfplumber.open(pdf_path) as pdf:
#                 total_pages = len(pdf.pages)
#                 print(f"Processing {total_pages} pages...")
                
#                 for page_num, page in enumerate(pdf.pages):
#                     print(f"\rपृष्ठ {page_num + 1}/{total_pages}", end="", flush=True)
                    
#                     page_text = page.extract_text()
#                     if page_text:
#                         # Clean the text to remove English and numbers
#                         cleaned_text = self.clean_nepali_text(page_text)
#                         if cleaned_text.strip():  # Only add if there's content after cleaning
#                             text += self.format_page_separator(page_num + 1) + cleaned_text
                    
#                     # Clear memory every few pages
#                     if (page_num + 1) % 10 == 0:
#                         gc.collect()
                
#                 print()  # New line after progress
#                 return text
                
#         except Exception as e:
#             print(f"Error with text extraction: {e}")
#             return None
    
#     def extract_with_batched_ocr(self, pdf_path, output_file=None, language='nep'):
#         """
#         Memory-efficient OCR - processes pages in small batches
#         Optionally saves directly to file to avoid keeping everything in memory
#         """
#         try:
#             # Get total page count first
#             from pdf2image.exceptions import PDFInfoNotInstalledError
            
#             print("Analyzing PDF...")
#             total_pages = self._get_page_count(pdf_path)
#             print(f"Total pages: {total_pages}")
            
#             # Open output file if specified
#             output_handle = None
#             if output_file:
#                 output_handle = open(output_file, 'w', encoding='utf-8')
#                 print(f"फाइलमा सेभ गर्दै: {output_file}")
            
#             all_text = ""
            
#             # Process in batches
#             for start_page in range(0, total_pages, self.max_pages_batch):
#                 end_page = min(start_page + self.max_pages_batch, total_pages)
#                 print(f"\nपृष्ठ {start_page + 1}-{end_page} प्रक्रिया गर्दै...")
                
#                 # Convert only current batch to images
#                 images = convert_from_path(
#                     pdf_path,
#                     dpi=self.dpi,
#                     first_page=start_page + 1,
#                     last_page=end_page
#                 )
                
#                 # Process each image in the batch
#                 for i, image in enumerate(images):
#                     current_page = start_page + i + 1
#                     print(f"  पृष्ठ {current_page} OCR...")
                    
#                     # Extract text from current page
#                     try:
#                         page_text = pytesseract.image_to_string(
#                             image, 
#                             lang=language,
#                             config='--psm 6'
#                         )
                        
#                         # Clean the extracted text
#                         cleaned_text = self.clean_nepali_text(page_text)
                        
#                         if cleaned_text.strip():  # Only add if there's meaningful content
#                             formatted_text = self.format_page_separator(current_page) + cleaned_text
                            
#                             if output_handle:
#                                 # Write directly to file
#                                 output_handle.write(formatted_text)
#                                 output_handle.flush()
#                             else:
#                                 all_text += formatted_text
                            
#                     except Exception as e:
#                         print(f"पृष्ठ {current_page} त्रुटि: {e}")
                    
#                     # Clear individual image from memory
#                     del image
                
#                 # Clear batch from memory
#                 del images
#                 gc.collect()
                
#                 print(f"  पूर्ण भयो। मेमोरी खाली गरियो।")
            
#             # Close output file if used
#             if output_handle:
#                 output_handle.close()
#                 print(f"\nटेक्स्ट सेभ भयो: {output_file}")
#                 return f"टेक्स्ट सेभ भयो {output_file}"
#             else:
#                 return all_text
                
#         except Exception as e:
#             print(f"Error with batched OCR: {e}")
#             if output_handle:
#                 output_handle.close()
#             return None
    
#     def _get_page_count(self, pdf_path):
#         """Get total page count without loading entire PDF"""
#         try:
#             with pdfplumber.open(pdf_path) as pdf:
#                 return len(pdf.pages)
#         except:
#             # Fallback method
#             try:
#                 with open(pdf_path, 'rb') as file:
#                     pdf_reader = PyPDF2.PdfReader(file)
#                     return len(pdf_reader.pages)
#             except:
#                 return 1  # Assume 1 page if can't determine
    
#     def extract_single_page(self, pdf_path, page_number, language='nep'):
#         """Extract text from a single page (for testing)"""
#         try:
#             print(f"पृष्ठ {page_number} निकाल्दै...")
            
#             # Convert single page to image
#             images = convert_from_path(
#                 pdf_path,
#                 dpi=self.dpi,
#                 first_page=page_number,
#                 last_page=page_number
#             )
            
#             if images:
#                 image = images[0]
#                 text = pytesseract.image_to_string(image, lang=language, config='--psm 6')
                
#                 # Clean the text
#                 cleaned_text = self.clean_nepali_text(text)
                
#                 # Clean up
#                 del images
#                 del image
#                 gc.collect()
                
#                 return cleaned_text
            
#         except Exception as e:
#             print(f"एउटै पृष्ठ निकाल्न त्रुटि: {e}")
#             return None
    
#     def smart_extract(self, pdf_path, output_file=None, language='nep'):
#         """
#         Smart extraction: tries text extraction first, falls back to OCR
#         Saves directly to file to minimize memory usage
#         """
#         print("=== नेपाली PDF टेक्स्ट निकाल्ने ===")
#         print(f"इनपुट: {pdf_path}")
        
#         # First, try regular text extraction
#         print("\n१. सामान्य टेक्स्ट निकाल्ने प्रयास...")
#         text = self.extract_text_lightweight(pdf_path)
        
#         if text and self._contains_meaningful_text(text):
#             print("✓ सामान्य निकासीमा अर्थपूर्ण टेक्स्ट भेटियो!")
            
#             if output_file:
#                 with open(output_file, 'w', encoding='utf-8') as f:
#                     f.write(text)
#                 print(f"✓ टेक्स्ट सेभ भयो: {output_file}")
#                 return f"टेक्स्ट सेभ भयो {output_file}"
#             else:
#                 return text
        
#         # If regular extraction didn't work, use OCR
#         print("\n२. सामान्य निकासी राम्रो भएन। OCR प्रयोग गर्दै...")
#         return self.extract_with_batched_ocr(pdf_path, output_file, language)
    
#     def _contains_meaningful_text(self, text):
#         """Check if extracted text is meaningful"""
#         if not text:
#             return False
        
#         # Remove whitespace and count actual characters
#         clean_text = ''.join(text.split())
        
#         # Check for minimum length and variety
#         if len(clean_text) < 50:
#             return False
        
#         # Check for Nepali characters (Devanagari)
#         nepali_chars = sum(1 for char in clean_text if 0x0900 <= ord(char) <= 0x097F)
        
#         # If we have Nepali characters or substantial English text
#         return nepali_chars > 10 or len(clean_text) > 200


# # Simple wrapper functions for easy use
# def extract_nepali_pdf_memory_safe(pdf_path, output_file=None, batch_size=3, dpi=200):
#     """
#     Memory-safe Nepali PDF extraction - removes all English text and numbers
    
#     Args:
#         pdf_path: Path to PDF file
#         output_file: Save to file instead of returning text (recommended for large PDFs)
#         batch_size: Pages to process at once (lower = less memory)
#         dpi: Image resolution (lower = less memory)
#     """
#     extractor = MemoryOptimizedPDFExtractor(max_pages_batch=batch_size, dpi=dpi)
#     return extractor.smart_extract(pdf_path, output_file, language='nep')


# def test_single_page_clean(pdf_path, page_num=1):
#     """Test extraction on a single page first - only Nepali text"""
#     extractor = MemoryOptimizedPDFExtractor(dpi=200)
#     return extractor.extract_single_page(pdf_path, page_num, language='nep')


# # Example usage
# if __name__ == "__main__":
#     # Configuration for low memory usage
#     PDF_PATH =  'Nepali Materials/अर्थतन्त्रमा कोरोना कहर.pdf'
    
#     filename = os.path.basename(PDF_PATH)
    
#     OUTPUT_FILE = f"Books/{filename}.csv"  # Output file
    
    
#     if not os.path.exists(PDF_PATH):
#         print("कृपया PDF_PATH लाई आफ्नो वास्तविक PDF फाइल पाथसँग अपडेट गर्नुहोस्")
#         sys.exit(1)
    
#     print("मेमोरी-अनुकूलित नेपाली PDF निकासी")
#     print(f"ब्याच साइज: 3 पृष्ठहरू (कम मेमोरी प्रयोग)")
#     print(f"DPI: 200 (गुणस्तर र मेमोरीको सन्तुलन)")
    
#     # Test single page first
#     print("\n=== पहिले एउटै पृष्ठ परीक्षण ===")
#     test_text = test_single_page_clean(PDF_PATH, 1)
#     if test_text:
#         print("✓ एउटै पृष्ठ परीक्षण सफल!")
#         print(f"नमूना: {test_text[:200]}...")
        
#         # Proceed with full extraction
#         print(f"\n=== पूर्ण PDF प्रक्रिया ===")
#         result = extract_nepali_pdf_memory_safe(
#             PDF_PATH, 
#             OUTPUT_FILE, 
#             batch_size=2,  # Very small batches
#             dpi=200        # Lower resolution
#         )
#         print(f"परिणाम: {result}")
#     else:
#         print("✗ एउटै पृष्ठ परीक्षण असफल")

In [ ]:
# import PyPDF2
# import pdfplumber
# import pytesseract
# from pdf2image import convert_from_path
# from PIL import Image
# import os
# import gc
# import sys
# import re
# import time
# from pathlib import Path

# class MemoryOptimizedPDFExtractor:
#     def clean_nepali_text(self, text):
#         """
#         Remove all English letters, numbers, and keep only Nepali/Devanagari text
#         """
#         if not text:
#             return ""
        
#         # Remove English letters (a-z, A-Z) and numbers (0-9)
#         # Keep Devanagari characters (0x0900-0x097F), whitespace, and common punctuation
#         cleaned_lines = []
        
#         for line in text.split('\n'):
#             # Remove English letters and numbers
#             cleaned_line = re.sub(r'[a-zA-Z0-9]', '', line)
            
#             # Remove extra whitespace
#             cleaned_line = ' '.join(cleaned_line.split())
            
#             # Only keep lines that have meaningful content after cleaning
#             if cleaned_line.strip():
#                 cleaned_lines.append(cleaned_line)
        
#         return '\n'.join(cleaned_lines)
    
#     def format_page_separator(self, page_number):
#         """Create page separator in Nepali numerals"""
#         # Convert to Nepali numerals
#         nepali_numerals = {'0': '०', '1': '१', '2': '२', '3': '३', '4': '४', 
#                           '5': '५', '6': '६', '7': '७', '8': '८', '9': '९'}
        
#         page_str = str(page_number)
#         nepali_page_num = ''.join(nepali_numerals.get(digit, digit) for digit in page_str)
        
#         return f"\n--- पृष्ठ {nepali_page_num} ---\n"
    
#     def __init__(self, max_pages_batch=5, dpi=200):
#         """
#         Memory-optimized PDF text extractor
        
#         Args:
#             max_pages_batch: Process this many pages at once (lower = less memory)
#             dpi: Image resolution (lower = less memory, but worse OCR quality)
#         """
#         self.max_pages_batch = max_pages_batch
#         self.dpi = dpi
        
#     def extract_text_lightweight(self, pdf_path):
#         """Lightweight text extraction - tries pdfplumber first"""
#         try:
#             text = ""
#             with pdfplumber.open(pdf_path) as pdf:
#                 total_pages = len(pdf.pages)
#                 print(f"Processing {total_pages} pages...")
                
#                 for page_num, page in enumerate(pdf.pages):
#                     print(f"\rपृष्ठ {page_num + 1}/{total_pages}", end="", flush=True)
                    
#                     page_text = page.extract_text()
#                     if page_text:
#                         # Clean the text to remove English and numbers
#                         cleaned_text = self.clean_nepali_text(page_text)
#                         if cleaned_text.strip():  # Only add if there's content after cleaning
#                             text += self.format_page_separator(page_num + 1) + cleaned_text
                    
#                     # Clear memory every few pages
#                     if (page_num + 1) % 10 == 0:
#                         gc.collect()
                
#                 print()  # New line after progress
#                 return text
                
#         except Exception as e:
#             print(f"Error with text extraction: {e}")
#             return None
    
#     def extract_with_batched_ocr(self, pdf_path, output_file=None, language='nep'):
#         """
#         Memory-efficient OCR - processes pages in small batches
#         Optionally saves directly to file to avoid keeping everything in memory
#         """
#         try:
#             # Get total page count first
#             from pdf2image.exceptions import PDFInfoNotInstalledError
            
#             print("Analyzing PDF...")
#             total_pages = self._get_page_count(pdf_path)
#             print(f"Total pages: {total_pages}")
            
#             # Open output file if specified
#             output_handle = None
#             if output_file:
#                 output_handle = open(output_file, 'w', encoding='utf-8')
#                 print(f"फाइलमा सेभ गर्दै: {output_file}")
            
#             all_text = ""
            
#             # Process in batches
#             for start_page in range(0, total_pages, self.max_pages_batch):
#                 end_page = min(start_page + self.max_pages_batch, total_pages)
#                 print(f"\nपृष्ठ {start_page + 1}-{end_page} प्रक्रिया गर्दै...")
                
#                 # Convert only current batch to images
#                 images = convert_from_path(
#                     pdf_path,
#                     dpi=self.dpi,
#                     first_page=start_page + 1,
#                     last_page=end_page
#                 )
                
#                 # Process each image in the batch
#                 for i, image in enumerate(images):
#                     current_page = start_page + i + 1
#                     print(f"  पृष्ठ {current_page} OCR...")
                    
#                     # Extract text from current page
#                     try:
#                         page_text = pytesseract.image_to_string(
#                             image, 
#                             lang=language,
#                             config='--psm 6'
#                         )
                        
#                         # Clean the extracted text
#                         cleaned_text = self.clean_nepali_text(page_text)
                        
#                         if cleaned_text.strip():  # Only add if there's meaningful content
#                             formatted_text = self.format_page_separator(current_page) + cleaned_text
                            
#                             if output_handle:
#                                 # Write directly to file
#                                 output_handle.write(formatted_text)
#                                 output_handle.flush()
#                             else:
#                                 all_text += formatted_text
                            
#                     except Exception as e:
#                         print(f"पृष्ठ {current_page} त्रुटि: {e}")
                    
#                     # Clear individual image from memory
#                     del image
                
#                 # Clear batch from memory
#                 del images
#                 gc.collect()
                
#                 print(f"  पूर्ण भयो। मेमोरी खाली गरियो।")
            
#             # Close output file if used
#             if output_handle:
#                 output_handle.close()
#                 print(f"\nटेक्स्ट सेभ भयो: {output_file}")
#                 return f"टेक्स्ट सेभ भयो {output_file}"
#             else:
#                 return all_text
                
#         except Exception as e:
#             print(f"Error with batched OCR: {e}")
#             if output_handle:
#                 output_handle.close()
#             return None
    
#     def _get_page_count(self, pdf_path):
#         """Get total page count without loading entire PDF"""
#         try:
#             with pdfplumber.open(pdf_path) as pdf:
#                 return len(pdf.pages)
#         except:
#             # Fallback method
#             try:
#                 with open(pdf_path, 'rb') as file:
#                     pdf_reader = PyPDF2.PdfReader(file)
#                     return len(pdf_reader.pages)
#             except:
#                 return 1  # Assume 1 page if can't determine
    
#     def extract_single_page(self, pdf_path, page_number, language='nep'):
#         """Extract text from a single page (for testing)"""
#         try:
#             print(f"पृष्ठ {page_number} निकाल्दै...")
            
#             # Convert single page to image
#             images = convert_from_path(
#                 pdf_path,
#                 dpi=self.dpi,
#                 first_page=page_number,
#                 last_page=page_number
#             )
            
#             if images:
#                 image = images[0]
#                 text = pytesseract.image_to_string(image, lang=language, config='--psm 6')
                
#                 # Clean the text
#                 cleaned_text = self.clean_nepali_text(text)
                
#                 # Clean up
#                 del images
#                 del image
#                 gc.collect()
                
#                 return cleaned_text
            
#         except Exception as e:
#             print(f"एउटै पृष्ठ निकाल्न त्रुटि: {e}")
#             return None
    
#     def smart_extract(self, pdf_path, output_file=None, language='nep'):
#         """
#         Smart extraction: tries text extraction first, falls back to OCR
#         Saves directly to file to minimize memory usage
#         """
#         print("=== नेपाली PDF टेक्स्ट निकाल्ने ===")
#         print(f"इनपुट: {pdf_path}")
        
#         # First, try regular text extraction
#         print("\n१. सामान्य टेक्स्ट निकाल्ने प्रयास...")
#         text = self.extract_text_lightweight(pdf_path)
        
#         if text and self._contains_meaningful_text(text):
#             print("✓ सामान्य निकासीमा अर्थपूर्ण टेक्स्ट भेटियो!")
            
#             if output_file:
#                 with open(output_file, 'w', encoding='utf-8') as f:
#                     f.write(text)
#                 print(f"✓ टेक्स्ट सेभ भयो: {output_file}")
#                 return f"टेक्स्ट सेभ भयो {output_file}"
#             else:
#                 return text
        
#         # If regular extraction didn't work, use OCR
#         print("\n२. सामान्य निकासी राम्रो भएन। OCR प्रयोग गर्दै...")
#         return self.extract_with_batched_ocr(pdf_path, output_file, language)
    
#     def _contains_meaningful_text(self, text):
#         """Check if extracted text is meaningful"""
#         if not text:
#             return False
        
#         # Remove whitespace and count actual characters
#         clean_text = ''.join(text.split())
        
#         # Check for minimum length and variety
#         if len(clean_text) < 50:
#             return False
        
#         # Check for Nepali characters (Devanagari)
#         nepali_chars = sum(1 for char in clean_text if 0x0900 <= ord(char) <= 0x097F)
        
#         # If we have Nepali characters or substantial English text
#         return nepali_chars > 10 or len(clean_text) > 200


# class BatchPDFProcessor:
#     """
#     Batch processor for multiple PDF files in a directory
#     """
    
#     def __init__(self, batch_size=3, dpi=200):
#         self.extractor = MemoryOptimizedPDFExtractor(max_pages_batch=batch_size, dpi=dpi)
#         self.processed_files = []
#         self.failed_files = []
    
#     def find_pdf_files(self, folder_path):
#         """Find all PDF files in the given folder"""
#         folder_path = Path(folder_path)
        
#         if not folder_path.exists():
#             raise FileNotFoundError(f"Folder not found: {folder_path}")
        
#         if not folder_path.is_dir():
#             raise NotADirectoryError(f"Path is not a directory: {folder_path}")
        
#         # Find all PDF files (case insensitive)
#         pdf_files = []
#         for ext in ['*.pdf', '*.PDF']:
#             pdf_files.extend(folder_path.glob(ext))
        
#         return sorted(pdf_files)
    
#     def process_folder(self, input_folder, output_folder=None, skip_existing=True):
#         """
#         Process all PDF files in a folder
        
#         Args:
#             input_folder: Path to folder containing PDF files
#             output_folder: Output folder (if None, creates 'extracted_text' in input folder)
#             skip_existing: Skip files that already have output files
#         """
#         input_path = Path(input_folder)
        
#         # Set up output folder
#         if output_folder is None:
#             output_path = input_path / "extracted_text"
#         else:
#             output_path = Path(output_folder)
        
#         # Create output folder if it doesn't exist
#         output_path.mkdir(parents=True, exist_ok=True)
        
#         # Find all PDF files
#         pdf_files = self.find_pdf_files(input_folder)
        
#         if not pdf_files:
#             print(f"कुनै PDF फाइलहरू भेटिएनन्: {input_folder}")
#             return
        
#         print(f"भेटिएका PDF फाइलहरू: {len(pdf_files)}")
#         print("=" * 50)
        
#         # Process each PDF
#         for i, pdf_file in enumerate(pdf_files, 1):
#             print(f"\n[{i}/{len(pdf_files)}] प्रक्रिया गर्दै: {pdf_file.name}")
            
#             # Generate output filename
#             output_filename = pdf_file.stem + "_extracted.txt"
#             output_file = output_path / output_filename
            
#             # Skip if output already exists and skip_existing is True
#             if skip_existing and output_file.exists():
#                 print(f"⏭️  स्किप गर्दै (पहिले नै अवस्थित): {output_filename}")
#                 continue
            
#             try:
#                 # Process the PDF
#                 start_time = time.time()
#                 result = self.extractor.smart_extract(
#                     str(pdf_file), 
#                     str(output_file), 
#                     language='nep'
#                 )
                
#                 end_time = time.time()
#                 duration = end_time - start_time
                
#                 if result:
#                     print(f"✅ सफल: {pdf_file.name} ({duration:.1f} सेकेन्ड)")
#                     self.processed_files.append({
#                         'input': str(pdf_file),
#                         'output': str(output_file),
#                         'duration': duration
#                     })
#                 else:
#                     print(f"❌ असफल: {pdf_file.name}")
#                     self.failed_files.append(str(pdf_file))
                
#             except Exception as e:
#                 print(f"❌ त्रुटि प्रक्रिया गर्दा {pdf_file.name}: {e}")
#                 self.failed_files.append(str(pdf_file))
            
#             # Clean memory between files
#             gc.collect()
        
#         # Print summary
#         self.print_summary()
    
#     def print_summary(self):
#         """Print processing summary"""
#         print("\n" + "=" * 50)
#         print("प्रक्रिया सारांश:")
#         print(f"✅ सफल फाइलहरू: {len(self.processed_files)}")
#         print(f"❌ असफल फाइलहरू: {len(self.failed_files)}")
        
#         if self.processed_files:
#             total_time = sum(f['duration'] for f in self.processed_files)
#             print(f"⏱️  कुल समय: {total_time:.1f} सेकेन्ड")
#             print(f"📁 आउटपुट फोल्डर: {Path(self.processed_files[0]['output']).parent}")
        
#         if self.failed_files:
#             print("\n❌ असफल फाइलहरू:")
#             for failed_file in self.failed_files:
#                 print(f"   - {Path(failed_file).name}")


# # Simple wrapper functions for easy use
# def extract_nepali_pdf_memory_safe(pdf_path, output_file=None, batch_size=3, dpi=200):
#     """
#     Memory-safe Nepali PDF extraction - removes all English text and numbers
    
#     Args:
#         pdf_path: Path to PDF file
#         output_file: Save to file instead of returning text (recommended for large PDFs)
#         batch_size: Pages to process at once (lower = less memory)
#         dpi: Image resolution (lower = less memory)
#     """
#     extractor = MemoryOptimizedPDFExtractor(max_pages_batch=batch_size, dpi=dpi)
#     return extractor.smart_extract(pdf_path, output_file, language='nep')


# def process_pdf_folder(input_folder, output_folder=None, batch_size=3, dpi=200, skip_existing=True):
#     """
#     Process all PDF files in a folder
    
#     Args:
#         input_folder: Path to folder containing PDF files
#         output_folder: Output folder (if None, creates 'extracted_text' in input folder)
#         batch_size: Pages to process at once (lower = less memory)
#         dpi: Image resolution (lower = less memory)
#         skip_existing: Skip files that already have output files
#     """
#     processor = BatchPDFProcessor(batch_size=batch_size, dpi=dpi)
#     processor.process_folder(input_folder, output_folder, skip_existing)
#     return processor


# def test_single_page_clean(pdf_path, page_num=1):
#     """Test extraction on a single page first - only Nepali text"""
#     extractor = MemoryOptimizedPDFExtractor(dpi=200)
#     return extractor.extract_single_page(pdf_path, page_num, language='nep')


# # Add these improved methods to your existing class

# def test_ocr_setup(self):
#     """Test if Nepali OCR is working properly"""
#     try:
#         # Get available languages
#         languages = pytesseract.get_languages()
#         print("Available Tesseract languages:")
#         for lang in languages:
#             print(f"  - {lang}")
        
#         if 'nep' in languages:
#             print("✅ Nepali (nep) language pack found!")
#             return True
#         else:
#             print("❌ Nepali language pack NOT found!")
#             print("Install with: sudo apt-get install tesseract-ocr-nep")
#             return False
            
#     except Exception as e:
#         print(f"Error checking OCR setup: {e}")
#         return False

# def extract_with_multiple_methods(self, pdf_path, page_number=1):
#     """Try multiple OCR methods for better results"""
#     try:
#         print(f"Testing multiple OCR methods on page {page_number}...")
        
#         # Convert page to image
#         images = convert_from_path(
#             pdf_path,
#             dpi=300,  # Higher DPI for better quality
#             first_page=page_number,
#             last_page=page_number
#         )
        
#         if not images:
#             return None
        
#         image = images[0]
        
#         # Method 1: Standard Nepali OCR
#         print("Method 1: Standard Nepali OCR...")
#         try:
#             text1 = pytesseract.image_to_string(image, lang='nep', config='--psm 6')
#             print(f"Result 1 (first 100 chars): {text1[:100]}")
#         except:
#             text1 = ""
        
#         # Method 2: Hindi OCR (sometimes works better for Devanagari)
#         print("Method 2: Hindi OCR...")
#         try:
#             text2 = pytesseract.image_to_string(image, lang='hin', config='--psm 6')
#             print(f"Result 2 (first 100 chars): {text2[:100]}")
#         except:
#             text2 = ""
        
#         # Method 3: Multiple languages
#         print("Method 3: Multi-language OCR...")
#         try:
#             text3 = pytesseract.image_to_string(image, lang='nep+hin+eng', config='--psm 6')
#             print(f"Result 3 (first 100 chars): {text3[:100]}")
#         except:
#             text3 = ""
        
#         # Method 4: Different PSM mode
#         print("Method 4: Different page segmentation...")
#         try:
#             text4 = pytesseract.image_to_string(image, lang='nep', config='--psm 1')
#             print(f"Result 4 (first 100 chars): {text4[:100]}")
#         except:
#             text4 = ""
        
#         # Clean up
#         del images, image
#         gc.collect()
        
#         return {
#             'standard_nepali': text1,
#             'hindi': text2,
#             'multi_lang': text3,
#             'different_psm': text4
#         }
        
#     except Exception as e:
#         print(f"Error in multiple methods test: {e}")
#         return None

# def preprocess_image_for_ocr(self, image):
#     """Preprocess image to improve OCR accuracy"""
#     import cv2
#     import numpy as np
    
#     try:
#         # Convert PIL to OpenCV format
#         opencv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
#         # Convert to grayscale
#         gray = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2GRAY)
        
#         # Apply denoising
#         denoised = cv2.fastNlMeansDenoising(gray)
        
#         # Apply threshold to get better contrast
#         _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
#         # Convert back to PIL format
#         processed_image = Image.fromarray(thresh)
        
#         return processed_image
        
#     except ImportError:
#         print("OpenCV not installed. Using original image.")
#         return image
#     except Exception as e:
#         print(f"Error preprocessing image: {e}")
#         return image

# def extract_with_preprocessing(self, pdf_path, page_number=1):
#     """Extract text with image preprocessing"""
#     try:
#         print(f"Extracting with preprocessing - Page {page_number}")
        
#         # Convert to image
#         images = convert_from_path(
#             pdf_path,
#             dpi=300,
#             first_page=page_number,
#             last_page=page_number
#         )
        
#         if not images:
#             return None
        
#         original_image = images[0]
        
#         # Preprocess image
#         processed_image = self.preprocess_image_for_ocr(original_image)
        
#         # Try OCR on both original and processed
#         print("Trying OCR on original image...")
#         text_original = pytesseract.image_to_string(original_image, lang='nep', config='--psm 6')
        
#         print("Trying OCR on processed image...")
#         text_processed = pytesseract.image_to_string(processed_image, lang='nep', config='--psm 6')
        
#         # Clean up
#         del images, original_image, processed_image
#         gc.collect()
        
#         print(f"Original result (first 100): {text_original[:100]}")
#         print(f"Processed result (first 100): {text_processed[:100]}")
        
#         # Return the better result
#         if len(text_processed.strip()) > len(text_original.strip()):
#             return text_processed
#         else:
#             return text_original
            
#     except Exception as e:
#         print(f"Error with preprocessing: {e}")
#         return None

# # Updated diagnostic function
# def diagnose_pdf_issues(pdf_path, test_page=1):
#     """Comprehensive diagnosis of PDF extraction issues"""
#     extractor = MemoryOptimizedPDFExtractor()
    
#     print("🔍 PDF निदान शुरू गर्दै...")
#     print("=" * 50)
    
#     # Check if file exists
#     if not os.path.exists(pdf_path):
#         print(f"❌ फाइल भेटिएन: {pdf_path}")
#         return
    
#     print(f"📁 फाइल: {pdf_path}")
#     print(f"📄 परीक्षण पृष्ठ: {test_page}")
    
#     # Test OCR setup
#     print("\n1. OCR सेटअप जाँच...")
#     ocr_ok = extractor.test_ocr_setup()
    
#     if not ocr_ok:
#         print("⚠️  Nepali OCR सेटअप समस्या छ!")
#         return
    
#     # Test regular text extraction
#     print("\n2. सामान्य टेक्स्ट निकासी...")
#     regular_text = extractor.extract_text_lightweight(pdf_path)
#     if regular_text and len(regular_text.strip()) > 50:
#         print("✅ सामान्य निकासी सफल!")
#         print(f"नमूना: {regular_text[:200]}...")
#         return
#     else:
#         print("❌ सामान्य निकासी असफल। OCR आवश्यक।")
    
#     # Test multiple OCR methods
#     print(f"\n3. विभिन्न OCR विधिहरू (पृष्ठ {test_page})...")
#     methods_result = extractor.extract_with_multiple_methods(pdf_path, test_page)
    
#     if methods_result:
#         for method, result in methods_result.items():
#             quality = "✅ राम्रो" if len(result.strip()) > 20 else "❌ खराब"
#             print(f"{method}: {quality} ({len(result)} chars)")
    
#     # Test with preprocessing
#     print(f"\n4. इमेज प्रोसेसिंग सहित...")
#     preprocessed_result = extractor.extract_with_preprocessing(pdf_path, test_page)
#     if preprocessed_result:
#         quality = "✅ राम्रो" if len(preprocessed_result.strip()) > 20 else "❌ खराब"
#         print(f"प्रोसेसिंग परिणाम: {quality}")
#         print(f"नमूना: {preprocessed_result[:200]}...")
    
#     print("\n" + "=" * 50)
#     print("निदान सम्पन्न!")

# # # Test your specific file
# # if __name__ == "__main__":
# #     # Test your problematic PDF
# #     PDF_PATH = 'Nepali Materials/अर्थतन्त्रमा कोरोना कहर.pdf'
    
# #     # Run diagnosis
# #     diagnose_pdf_issues(PDF_PATH, test_page=1)
# # # Example usage
# if __name__ == "__main__":
#     # Configuration
#     INPUT_FOLDER = 'Nepali Materials'  # Folder containing PDF files
#     OUTPUT_FOLDER = 'Book'            # Output folder (optional)
    
#     # Method 1: Process entire folder
#     print("🚀 सबै PDF फाइलहरू प्रक्रिया गर्दै...")
#     processor = process_pdf_folder(
#         input_folder=INPUT_FOLDER,
#         output_folder=OUTPUT_FOLDER,
#         batch_size=2,        # Small batches for low memory
#         dpi=200,            # Balance of quality and speed
#         skip_existing=True   # Skip already processed files
#     )
    
#     # Method 2: Process specific folder with custom settings
#     # processor = BatchPDFProcessor(batch_size=1, dpi=150)  # Very low memory
#     # processor.process_folder(INPUT_FOLDER, OUTPUT_FOLDER, skip_existing=True)
    
#     print("\n🎉 सबै फाइलहरू प्रक्रिया सम्पन्न!")

In [ ]:
%pip install opencv-python

In [ ]:
import PyPDF2
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance, ImageFilter
import os
import gc
import sys
import re
import time
from pathlib import Path
import cv2
import numpy as np

class ScannedPDFExtractor:
    def __init__(self, max_pages_batch=3, dpi=300):
        """
        Optimized for scanned PDF documents
        
        Args:
            max_pages_batch: Process this many pages at once (lower = less memory)
            dpi: Image resolution (300+ recommended for scanned docs)
        """
        self.max_pages_batch = max_pages_batch
        self.dpi = dpi
        
    def clean_nepali_text(self, text):
        """Clean and filter text to keep only Nepali/Devanagari"""
        if not text:
            return ""
        
        cleaned_lines = []
        for line in text.split('\n'):
            # Remove English letters, numbers, and excessive punctuation
            cleaned_line = re.sub(r'[a-zA-Z0-9]', '', line)
            cleaned_line = re.sub(r'[^\u0900-\u097F\s।॥\-\,\.\?\!\:\;]', '', cleaned_line)
            cleaned_line = ' '.join(cleaned_line.split())
            
            if cleaned_line.strip() and len(cleaned_line.strip()) > 2:
                cleaned_lines.append(cleaned_line)
        
        return '\n'.join(cleaned_lines)
    
    def format_page_separator(self, page_number):
        """Create page separator in Nepali numerals"""
        nepali_numerals = {'0': '०', '1': '१', '2': '२', '3': '३', '4': '४', 
                          '5': '५', '6': '६', '7': '७', '8': '८', '9': '९'}
        
        page_str = str(page_number)
        nepali_page_num = ''.join(nepali_numerals.get(digit, digit) for digit in page_str)
        return f"\n--- पृष्ठ {nepali_page_num} ---\n"
    
    def enhance_scanned_image(self, image):
        """
        Enhance scanned image for better OCR results
        """
        try:
            # Convert PIL to OpenCV format
            opencv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
            
            # Convert to grayscale
            gray = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2GRAY)
            
            # Apply denoising to remove scanning artifacts
            denoised = cv2.fastNlMeansDenoising(gray, h=10, templateWindowSize=7, searchWindowSize=21)
            
            # Apply adaptive threshold for better text contrast
            adaptive_thresh = cv2.adaptiveThreshold(
                denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
            )
            
            # Optional: Apply morphological operations to clean text
            kernel = np.ones((1,1), np.uint8)
            cleaned = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)
            
            # Convert back to PIL
            enhanced_image = Image.fromarray(cleaned)
            
            return enhanced_image
            
        except ImportError:
            print("⚠️  OpenCV not available. Using PIL enhancement...")
            return self.enhance_with_pil(image)
        except Exception as e:
            print(f"Error enhancing image: {e}")
            return self.enhance_with_pil(image)
    
    def enhance_with_pil(self, image):
        """Fallback image enhancement using PIL only"""
        try:
            # Convert to grayscale
            if image.mode != 'L':
                image = image.convert('L')
            
            # Enhance contrast
            enhancer = ImageEnhance.Contrast(image)
            image = enhancer.enhance(2.0)
            
            # Enhance sharpness
            enhancer = ImageEnhance.Sharpness(image)
            image = enhancer.enhance(2.0)
            
            # Apply a slight blur to reduce noise, then sharpen
            image = image.filter(ImageFilter.MedianFilter(size=3))
            
            return image
            
        except Exception as e:
            print(f"PIL enhancement error: {e}")
            return image
    
    def extract_text_from_scanned_page(self, image, page_num, language='nep'):
        """
        Extract text from a single scanned page image with multiple methods
        """
        results = {}
        
        # Method 1: Original image with different PSM modes
        psm_modes = [6, 1, 3, 4, 8]  # Different page segmentation modes
        
        for psm in psm_modes:
            try:
                config = f'--psm {psm} -c tessedit_char_whitelist=अआइईउऊएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशषसह।॥ािीुूेैोौंः्०१२३४५६७८९ '
                text = pytesseract.image_to_string(image, lang=language, config=config)
                results[f'psm_{psm}'] = text
            except Exception as e:
                print(f"PSM {psm} failed: {e}")
                results[f'psm_{psm}'] = ""
        
        # Method 2: Enhanced image
        try:
            enhanced_image = self.enhance_scanned_image(image)
            config = '--psm 6 -c tessedit_char_whitelist=अआइईउऊएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशषसह।॥ािीुूेैोौंः्०१२३४५६७८९ '
            enhanced_text = pytesseract.image_to_string(enhanced_image, lang=language, config=config)
            results['enhanced'] = enhanced_text
        except Exception as e:
            print(f"Enhanced extraction failed: {e}")
            results['enhanced'] = ""
        
        # Method 3: Try different languages
        for lang in ['nep', 'hin', 'san']:
            try:
                text = pytesseract.image_to_string(image, lang=lang, config='--psm 6')
                results[f'lang_{lang}'] = text
            except:
                results[f'lang_{lang}'] = ""
        
        # Find the best result
        best_result = ""
        best_score = 0
        
        for method, text in results.items():
            if text:
                # Score based on Devanagari character count and length
                devanagari_chars = sum(1 for char in text if 0x0900 <= ord(char) <= 0x097F)
                score = devanagari_chars * 2 + len(text.strip())
                
                if score > best_score:
                    best_score = score
                    best_result = text
                    print(f"  📊 Best method so far: {method} (score: {score})")
        
        return best_result
    
    def extract_scanned_pdf(self, pdf_path, output_file=None, language='nep'):
        """
        Extract text from scanned PDF with optimized settings
        """
        try:
            print("=== स्क्यान भएको PDF टेक्स्ट निकाल्ने ===")
            print(f"फाइल: {pdf_path}")
            print(f"DPI: {self.dpi} (उच्च गुणस्तर)")
            
            # Get total pages
            total_pages = self._get_page_count(pdf_path)
            print(f"कुल पृष्ठहरू: {total_pages}")
            
            # Setup output
            output_handle = None
            if output_file:
                output_handle = open(output_file, 'w', encoding='utf-8')
                print(f"आउटपुट फाइल: {output_file}")
            
            all_text = ""
            
            # Process in batches
            for start_page in range(0, total_pages, self.max_pages_batch):
                end_page = min(start_page + self.max_pages_batch, total_pages)
                print(f"\n📄 पृष्ठ {start_page + 1}-{end_page} प्रक्रिया गर्दै...")
                
                # Convert batch to high-resolution images
                try:
                    images = convert_from_path(
                        pdf_path,
                        dpi=self.dpi,
                        first_page=start_page + 1,
                        last_page=end_page,
                        fmt='PNG'  # Better quality for scanned docs
                    )
                except Exception as e:
                    print(f"PDF to image conversion failed: {e}")
                    continue
                
                # Process each image
                for i, image in enumerate(images):
                    current_page = start_page + i + 1
                    print(f"  🔍 पृष्ठ {current_page} OCR गर्दै...")
                    
                    try:
                        # Extract text with multiple methods
                        start_time = time.time()
                        page_text = self.extract_text_from_scanned_page(image, current_page, language)
                        end_time = time.time()
                        
                        if page_text:
                            # Clean the text
                            cleaned_text = self.clean_nepali_text(page_text)
                            
                            if cleaned_text.strip():
                                formatted_text = self.format_page_separator(current_page) + cleaned_text
                                
                                if output_handle:
                                    output_handle.write(formatted_text)
                                    output_handle.flush()
                                else:
                                    all_text += formatted_text
                                
                                print(f"    ✅ सफल ({end_time-start_time:.1f}s, {len(cleaned_text)} chars)")
                            else:
                                print(f"    ⚠️  सफाई पछि कुनै टेक्स्ट बाँकी छैन")
                        else:
                            print(f"    ❌ कुनै टेक्स्ट भेटिएन")
                            
                    except Exception as e:
                        print(f"    ❌ त्रुटि: {e}")
                    
                    # Clear memory
                    del image
                
                # Clear batch
                del images
                gc.collect()
                print(f"  🧹 मेमोरी सफा गरियो")
            
            # Close output file
            if output_handle:
                output_handle.close()
                print(f"\n✅ टेक्स्ट सेभ भयो: {output_file}")
                return f"टेक्स्ट सेभ भयो {output_file}"
            else:
                return all_text
                
        except Exception as e:
            print(f"मुख्य त्रुटि: {e}")
            if output_handle:
                output_handle.close()
            return None
    
    def _get_page_count(self, pdf_path):
        """Get total page count"""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                return len(pdf.pages)
        except:
            try:
                with open(pdf_path, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    return len(pdf_reader.pages)
            except:
                return 1
    
    def test_single_scanned_page(self, pdf_path, page_number=1):
        """Test extraction on a single scanned page"""
        try:
            print(f"🧪 पृष्ठ {page_number} परीक्षण...")
            
            # Convert single page to high-res image
            images = convert_from_path(
                pdf_path,
                dpi=self.dpi,
                first_page=page_number,
                last_page=page_number,
                fmt='PNG'
            )
            
            if images:
                image = images[0]
                print(f"📏 इमेज साइज: {image.size}")
                
                # Extract text
                text = self.extract_text_from_scanned_page(image, page_number)
                cleaned = self.clean_nepali_text(text)
                
                # Cleanup
                del images, image
                gc.collect()
                
                print(f"✅ परीक्षण सम्पन्न!")
                print(f"📊 कच्चा टेक्स्ट: {len(text)} chars")
                print(f"📊 सफा टेक्स्ट: {len(cleaned)} chars")
                print(f"🔤 नमूना:\n{cleaned[:300]}...")
                
                return cleaned
            
        except Exception as e:
            print(f"परीक्षण त्रुटि: {e}")
            return None


class BatchScannedPDFProcessor:
    """Process multiple scanned PDFs"""
    
    def __init__(self, batch_size=2, dpi=300):
        self.extractor = ScannedPDFExtractor(max_pages_batch=batch_size, dpi=dpi)
        self.processed_files = []
        self.failed_files = []
    
    def process_scanned_folder(self, input_folder, output_folder=None, skip_existing=True):
        """Process all scanned PDFs in folder"""
        input_path = Path(input_folder)
        
        if output_folder is None:
            output_path = input_path / "extracted_scanned_text"
        else:
            output_path = Path(output_folder)
        
        output_path.mkdir(parents=True, exist_ok=True)
        
        # Find PDFs
        pdf_files = []
        for ext in ['*.pdf', '*.PDF']:
            pdf_files.extend(input_path.glob(ext))
        
        if not pdf_files:
            print(f"कुनै PDF फाइल भेटिएन: {input_folder}")
            return
        
        print(f"🔍 भेटिएका स्क्यान PDF हरू: {len(pdf_files)}")
        print("=" * 60)
        
        for i, pdf_file in enumerate(pdf_files, 1):
            print(f"\n[{i}/{len(pdf_files)}] प्रक्रिया: {pdf_file.name}")
            
            output_filename = pdf_file.stem + "_scanned_extracted.txt"
            output_file = output_path / output_filename
            
            if skip_existing and output_file.exists():
                print(f"⏭️  स्किप (पहिले देखि नै छ): {output_filename}")
                continue
            
            try:
                start_time = time.time()
                result = self.extractor.extract_scanned_pdf(
                    str(pdf_file), 
                    str(output_file)
                )
                end_time = time.time()
                
                if result:
                    duration = end_time - start_time
                    print(f"✅ सफल: {pdf_file.name} ({duration:.1f}s)")
                    self.processed_files.append({
                        'input': str(pdf_file),
                        'output': str(output_file),
                        'duration': duration
                    })
                else:
                    print(f"❌ असफल: {pdf_file.name}")
                    self.failed_files.append(str(pdf_file))
                    
            except Exception as e:
                print(f"❌ त्रुटि {pdf_file.name}: {e}")
                self.failed_files.append(str(pdf_file))
            
            gc.collect()
        
        self.print_summary()
    
    def print_summary(self):
        """Print processing summary"""
        print("\n" + "=" * 60)
        print("स्क्यान PDF प्रक्रिया सारांश:")
        print(f"✅ सफल: {len(self.processed_files)}")
        print(f"❌ असफल: {len(self.failed_files)}")
        
        if self.processed_files:
            total_time = sum(f['duration'] for f in self.processed_files)
            print(f"⏱️  कुल समय: {total_time:.1f} सेकेन्ड")
            avg_time = total_time / len(self.processed_files)
            print(f"📊 औसत समय प्रति फाइल: {avg_time:.1f} सेकेन्ड")


# Simple functions for easy use
def extract_scanned_pdf(pdf_path, output_file=None, dpi=300, batch_size=2):
    """Extract text from scanned PDF"""
    extractor = ScannedPDFExtractor(max_pages_batch=batch_size, dpi=dpi)
    return extractor.extract_scanned_pdf(pdf_path, output_file)

def test_scanned_page(pdf_path, page_num=1, dpi=300):
    """Test single page extraction"""
    extractor = ScannedPDFExtractor(dpi=dpi)
    return extractor.test_single_scanned_page(pdf_path, page_num)

def process_scanned_folder(input_folder, output_folder=None, dpi=300, batch_size=2):
    """Process all scanned PDFs in folder"""
    processor = BatchScannedPDFProcessor(batch_size=batch_size, dpi=dpi)
    processor.process_scanned_folder(input_folder, output_folder)
    return processor


# Usage example
if __name__ == "__main__":
    # Configuration for scanned PDFs
    INPUT_FOLDER = '/var/home/ramrshrcg/Downloads/nepali books/milena'
    OUTPUT_FOLDER = 'Books_Scanned'
    
    # Test single page first
    print("🧪 पहिले एक पृष्ठ परीक्षण...")
    test_pdf = 'Nepali Materials/अर्थतन्त्रमा कोरोना कहर.pdf'
    
    if os.path.exists(test_pdf):
        result = test_scanned_page(test_pdf, page_num=1, dpi=700)
        
        if result and len(result.strip()) > 20:
            print("✅ परीक्षण सफल! पूर्ण प्रक्रिया सुरु गर्दै...")
            
            # Process all scanned PDFs
            processor = process_scanned_folder(
                INPUT_FOLDER, 
                OUTPUT_FOLDER, 
                dpi=700,        # High resolution for scanned docs
                batch_size=1    # Very small batches for memory
            )
        else:
            print("❌ परीक्षण असफल। कृपया समस्या जाँच गर्नुहोस्।")
    else:
        print(f"❌ फाइल भेटिएन: {test_pdf}")
        print("कृपया PDF_PATH अपडेट गर्नुहोस्")